### A Look at the Data

In order to get a better understanding of the data we will be looking at throughout this lesson, let's take a look at some of the characteristics of the dataset.

First, let's read in the data and necessary libraries.

In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [24]:
b_cal = pd.read_csv('boston_calendar.csv')
b_list = pd.read_csv('boston_listings.csv')
b_rev = pd.read_csv('boston_reviews.csv')

s_cal = pd.read_csv('seatle_calendar.csv')
s_list = pd.read_csv('seatle_listings.csv')
s_rev = pd.read_csv('seatle_reviews.csv')

## Step 1: Checking shape, nulls and columns with some minor changes (Basic cleaning)

> **Boston & Seatle Calendar**

### Check the sizes of cols and rows 

In [25]:
b_cal.head(2)

,listing_id,date,available,price
0,12147973,2017-09-05,f,NaN
1,12147973,2017-09-04,f,NaN


In [26]:
s_cal.head(2)

,listing_id,date,available,price
0,241032,2016-01-04,t,$85.00
1,241032,2016-01-05,t,$85.00


In [27]:
print("b_cal  :", b_cal.shape)
print("s_cal  :", s_cal.shape)

b_cal  : (1308890, 4)
s_cal  : (1393570, 4)


### Check Nulls

In [28]:
print(b_cal.info())
print('\n')
print("Null prop of price column: ", b_cal.price.isnull().sum()/b_cal.shape[0])
print("Proportion of False(unit unavailable at this date ):", b_cal.available[b_cal.available =='f' ].count()/b_cal.shape[0])

#Since the proportions are the same, then, all nans are when the units aren't available, let's be more sure
b_cal[b_cal['available']== 't']['price'].isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1308890 entries, 0 to 1308889
Data columns (total 4 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   listing_id  1308890 non-null  int64 
 1   date        1308890 non-null  object
 2   available   1308890 non-null  object
 3   price       643037 non-null   object
dtypes: int64(1), object(3)
memory usage: 39.9+ MB
None


Null prop of price column:  0.5087157820748879
Proportion of False(unit unavailable at this date ): 0.5087157820748879


0

In [29]:
print(s_cal.info())
print('\n')
print("Null prop of price column: ", s_cal.price.isnull().sum()/s_cal.shape[0])
print("Proportion of False(unit unavailable at this date ):", s_cal.available[s_cal.available =='f' ].count()/s_cal.shape[0])

#Since the proportions are the same, then, all nans are when the units aren't available, let's be more sure
s_cal[s_cal['available']== 't']['price'].isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1393570 entries, 0 to 1393569
Data columns (total 4 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   listing_id  1393570 non-null  int64 
 1   date        1393570 non-null  object
 2   available   1393570 non-null  object
 3   price       934542 non-null   object
dtypes: int64(1), object(3)
memory usage: 42.5+ MB
None


Null prop of price column:  0.32938998399793334
Proportion of False(unit unavailable at this date ): 0.32938998399793334


0

### Transfer Date column to datetime to ease manipulation, analysis and modeling. Check the period in which the data was collected. I create a dataframe with seperate date items and append it after

In [30]:
b_cal['date'] = pd.to_datetime(b_cal.date)
print(b_cal.date.dtype)
print('\n')

b_date_df = pd.DataFrame()
b_date_df['year'] = b_cal['date'].dt.year
b_date_df['month'] = b_cal['date'].dt.month
b_date_df['day'] =b_cal['date'].dt.day
b_date_df['dow'] =b_cal['date'].dt.strftime("%A")
b_cal = b_cal.join(b_date_df)
b_cal.head()

datetime64[ns]




,listing_id,date,available,price,year,month,day,dow
0,12147973,2017-09-05,f,NaN,2017,9,5,Tuesday
1,12147973,2017-09-04,f,NaN,2017,9,4,Monday
2,12147973,2017-09-03,f,NaN,2017,9,3,Sunday
3,12147973,2017-09-02,f,NaN,2017,9,2,Saturday
4,12147973,2017-09-01,f,NaN,2017,9,1,Friday


In [31]:
s_cal['date'] = pd.to_datetime(s_cal.date)
print(s_cal.date.dtype)
print('\n')

s_date_df = pd.DataFrame()
s_date_df['year'] = s_cal['date'].dt.year
s_date_df['month'] = s_cal['date'].dt.month
s_date_df['day'] =s_cal['date'].dt.day
s_date_df['dow'] =s_cal['date'].dt.strftime("%A")
s_cal = s_cal.join(s_date_df)
s_cal.head()

datetime64[ns]




,listing_id,date,available,price,year,month,day,dow
0,241032,2016-01-04,t,$85.00,2016,1,4,Monday
1,241032,2016-01-05,t,$85.00,2016,1,5,Tuesday
2,241032,2016-01-06,f,NaN,2016,1,6,Wednesday
3,241032,2016-01-07,f,NaN,2016,1,7,Thursday
4,241032,2016-01-08,f,NaN,2016,1,8,Friday


In [32]:
b_period =pd.DataFrame(b_cal.groupby(['year'], sort = True)['month'].value_counts())
b_period = b_period.rename(columns={'month':'count'}, level=0)
b_period = b_period.reset_index().sort_values(by=['year', 'month']).reset_index(drop = True)
b_period['obs_prop_per_month'] = np.round(b_period['count']/b_cal.shape[0]*100,1)
b_period.drop('count', inplace =True, axis = 1)
b_period

,year,month,obs_prop_per_month
0,2016,9,6.8
1,2016,10,8.5
2,2016,11,8.2
3,2016,12,8.5
4,2017,1,8.5
5,2017,2,7.7
6,2017,3,8.5
7,2017,4,8.2
8,2017,5,8.5
9,2017,6,8.2


In [36]:
s_period =pd.DataFrame(s_cal.groupby(['year'], sort = True)['month'].value_counts())
s_period = s_period.rename(columns={'month':'count'}, level=0)
s_period = s_period.reset_index().sort_values(by=['year', 'month']).reset_index(drop = True)
s_period['obs_prop_per_month'] = np.round(s_period['count']/s_cal.shape[0]*100,1)
s_period.drop('count', inplace =True, axis = 1)
s_period

,year,month,obs_prop_per_month
0,2016,1,7.7
1,2016,2,7.9
2,2016,3,8.5
3,2016,4,8.2
4,2016,5,8.5
5,2016,6,8.2
6,2016,7,8.5
7,2016,8,8.5
8,2016,9,8.2
9,2016,10,8.5


In [60]:
#Length of the first listing in both Boston and seatle Datasets
print(b_cal[b_cal['listing_id']== b_cal['listing_id'].iloc[0]].shape[0])
print(s_cal[s_cal['listing_id']== s_cal['listing_id'].iloc[0]].shape[0])

365

In [81]:
print("Number of unique Listing IDs in Boston Calendar: ", len(b_cal.listing_id.unique()))
print("Number of unique Listing IDs in Seatle Calendar: ", len(s_cal.listing_id.unique()))

Number of unique Listing IDs in Boston Calendar:  3585
Number of unique Listing IDs in Seatle Calendar:  3818


In [83]:
#For Boston Calendar
list_ids_not_year_long = []

for i in sorted(list(b_cal.listing_id.unique())):
    if b_cal[b_cal['listing_id']== i].shape[0] != 365:
        list_ids_not_year_long.append(i)
        
print(list_ids_not_year_long)
print('\n')
print(b_cal[b_cal['listing_id']== 12898806].shape[0]/2)

[12898806]


365.0


In [82]:
#For Seatle Calendar
list_ids_not_year_long = []

for i in sorted(list(s_cal.listing_id.unique())):
    if s_cal[s_cal['listing_id']== i].shape[0] != 365:
        list_ids_not_year_long.append(i)
list_ids_not_year_long

[]

/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
### Comments:  
[Boston & Seatle Calendar]
- The datasets have information about listing dates, availability and price tracked over a year for ever listing id
- There are no data entry errors, all nulls are due to the structuring of the Data (the listings that aren't available has no price)
<br><br>
- The Boston calendar Dataset ranges from `September'16` to `September'17`, No nulls 
- It has `1308890` rows and  `4` cols
- I added 4 cols that contain dateparts that will aid further analysis and model
- The Seatle calendar Dataset ranges from `January'16` to `January'17`, No nulls
- It has `1393570` rows and  `4` cols
- I added 4 cols that contain dateparts that will aid further analysis and model
<br><br>
- Number of unique Listing IDs in Boston Calendar:  `3585`
- Number of unique Listing IDs in Seatle Calendar:  `3818`
- Listing id `12898806` spans for 2 years in Boston Calendar

////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

 _______________________________________________________________________________________________________________________

## Step 1: Continue - 

> **Boston & Seatle Listings**

In [18]:
b_list.head(1)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,12147973,https://www.airbnb.com/rooms/12147973,20160906204935,2016-09-07,Sunny Bungalow in the City,"Cozy, sunny, family home. Master bedroom high...",The house has an open and cozy feel at the sam...,"Cozy, sunny, family home. Master bedroom high...",none,"Roslindale is quiet, convenient and friendly. ...",...,NaN,f,NaN,NaN,f,moderate,f,f,1,NaN


In [34]:
s_list.head(1)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,241032,https://www.airbnb.com/rooms/241032,20160104002432,2016-01-04,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,none,NaN,...,10.0,f,NaN,WASHINGTON,f,moderate,f,f,2,4.07


 ### Check the sizes of cols and rows 

In [94]:
print("Boston listings size :", b_list.shape)
print("Seatle listings size :", s_list.shape)
set_difference = set(b_list.columns) - set(s_list.columns)
print("Columns in Boston but not in Seatle:  ", set_difference)

Boston listings size : (3585, 95)
Seatle listings size : (3818, 92)
Columns in Boston but not in Seatle:   {'house_rules', 'access', 'interaction'}


In [93]:
b_list[['house_rules', 'access', 'interaction']].head(3)

,house_rules,access,interaction
0,Clean up and treat the home the way you'd like...,"You will have access to 2 bedrooms, a living r...",NaN
1,Pet friendly but please confirm with me if the...,Apt has one more bedroom (which I use) and lar...,"If I am at home, I am likely working in my hom..."
2,"I encourage you to use my kitchen, cooking and...","I am living in the apartment during your stay,...","ABOUT ME: I'm a laid-back, friendly, unmarried..."


### Check Nulls

In [115]:
print("Number of Non-null cols in Boston listings: ",  np.sum(b_list.isnull().sum()==0))
print("Number of Non-null cols in Seatle listings: ",  np.sum(s_list.isnull().sum()==0))

Number of Non-null cols in Boston listings:  51
Number of Non-null cols in Seatle listings:  47


In [123]:
# For Boston Listings The null proportion per column
print(b_list.isnull().sum()[b_list.isnull().sum()>0]/b_list.shape[0])

summary                         0.039888
space                           0.294840
neighborhood_overview           0.394700
notes                           0.550907
transit                         0.359833
access                          0.415342
interaction                     0.433473
house_rules                     0.332497
thumbnail_url                   0.167085
medium_url                      0.167085
xl_picture_url                  0.167085
host_location                   0.003068
host_about                      0.365132
host_response_time              0.131381
host_response_rate              0.131381
host_acceptance_rate            0.131381
host_neighbourhood              0.094561
neighbourhood                   0.151464
neighbourhood_group_cleansed    1.000000
city                            0.000558
zipcode                         0.010600
market                          0.003905
property_type                   0.000837
bathrooms                       0.003905
bedrooms        

In [120]:
# For Seatle Listings The null proportion per column
s_list.isnull().sum()[s_list.isnull().sum()>0]/s_list.shape[0]*100

summary                          4.635935
space                           14.903091
neighborhood_overview           27.029859
notes                           42.063908
transit                         24.463070
thumbnail_url                    8.381351
medium_url                       8.381351
xl_picture_url                   8.381351
host_name                        0.052383
host_since                       0.052383
host_location                    0.209534
host_about                      22.498690
host_response_time              13.698271
host_response_rate              13.698271
host_acceptance_rate            20.246202
host_is_superhost                0.052383
host_thumbnail_url               0.052383
host_picture_url                 0.052383
host_neighbourhood               7.857517
host_listings_count              0.052383
host_total_listings_count        0.052383
host_has_profile_pic             0.052383
host_identity_verified           0.052383
neighbourhood                   10

# Examine columns closely to prep them correctly!

/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
### Comments:  
[Boston & Seatle Listings]
- Boston listings size : `3585`, `95`
- Seatle listings size : `3818`, `92`
- Boston Listings has 3 cols that arent in Seatle, `house_rules`, `access`, `interaction`, types: `Object/String`
- Number of Non-null cols in Boston listings:  `51`, around half
- Number of Non-null cols in Seatle listings:  `47`, around half
